# Lenguaje de Configuración y Despliegue para Servicios (Estilo Ansible simplificado)
---

[URL](https://github.com/adramarilla/parseo1_2025)

## Objetivo

Realizar una herramienta de automatización para configuración de servidores y ejecutar acciones de manera declarativa

## Especificaciones léxicas

- **Palabras clave:**  
  `SERVIDOR`, `GRUPO`, `DESPLEGAR`, `ARCHIVO`, `PAQUETE`, `SERVICIO`,  
  `EJECUTAR`, `COPIAR_DESDE`, `EN`, `CON`, `VERIFICAR`, `SI`, `NO_EXISTE`

- **Identificadores:**  
  Nombres de servidores (`"web01"`, `"db_production"`),  
  nombres de grupos (`"webservers"`, `"databases"`),  
  nombres de paquetes (`"nginx"`, `"nodejs"`),  
  rutas (`"/var/www/app"`).

- **Literales:**  
  Strings (para comandos, rutas, contenido) y números (para puertos).

- **Operadores:**  
  `=`, `==`, `!=`, `>`, `<` (para comparaciones en verificaciones).

- **Símbolos:**  
  `{`, `}`, `[`, `]`, `(`, `)`, `:`, `-` (para bloques, listas y parámetros).

## Especificaciones sintácticas

```plaintext
GRUPO webservers {
    SERVIDOR "web01" DIRECCION = "192.168.1.10"
    SERVIDOR "web02" DIRECCION = "192.168.1.11"
}
```

```plaintext
DESPLEGAR mi_aplicacion EN webservers {

    # Garantizar que un paquete está instalado
    PAQUETE "nginx" DEBE_ESTAR_INSTALADO

    # Garantizar que un servicio está ejecutándose
    SERVICIO "nginx" DEBE_ESTAR_EN_EJECUCION

    # Copiar archivos locales al servidor
    COPIAR_DESDE "./app/*" HACIA "/var/www/html/"
}
```

## Especificaciones semánticas

- Verificar que los grupos y servidores referenciados en un `DESPLEGAR ... EN` existan
- **Análisis de tipos:** Asegurar que `DIRECCION` sea un *string* y que el puerto en `VERIFICAR` sea un número 
- **Chequeos de contexto:** No permitir `COPIAR_DESDE` fuera de un bloque `DESPLEGAR`
- **Tabla de símbolos:** Llevar un registro de todos los servidores, grupos y sus propiedades

## Lexer

In [ ]:
import ply.lex as lex

# -----------------------
# Palabras reservadas
# -----------------------
# Incluye palabras de control e iteración

reserved = {
    'VAR': 'VAR',
    'SERVIDOR': 'SERVIDOR',
    'GRUPO': 'GRUPO',
    'DESPLEGAR': 'DESPLEGAR',
    'ARCHIVO': 'ARCHIVO',
    'PAQUETE': 'PAQUETE',
    'SERVICIO': 'SERVICIO',
    'EJECUTAR': 'EJECUTAR',
    'COPIAR_DESDE': 'COPIAR_DESDE',
    'HACIA' : 'HACIA',
    'EN': 'EN',
    'CON': 'CON',
    'VERIFICAR': 'VERIFICAR',
    'NO_EXISTE': 'NO_EXISTE',
    'DIRECCION': 'DIRECCION',
    'HACER': 'HACER',
    'DEBE_ESTAR_INSTALADO': 'DEBE_ESTAR_INSTALADO',
    'DEBE_ESTAR_EN_EJECUCION': 'DEBE_ESTAR_EN_EJECUCION',

    'SI': 'SI',
    'SINO': 'SINO',
    'MIENTRAS': 'MIENTRAS', 

}

# -----------------------
# Tokens
# -----------------------
tokens = [
    'ID',
    'STRING',
    'NUMBER',

    # Operadores
    'EQUALS', 'EQ', 'NEQ', 'GT', 'LT',

    # Símbolos
    'LBRACE', 'RBRACE',
    'LBRACKET', 'RBRACKET',
    'LPAREN', 'RPAREN',
    'COLON', 'DASH',
] + list(reserved.values())

# -----------------------
# Expresiones regulares
# -----------------------

# Operadores
t_EQUALS = r'='
t_EQ = r'=='
t_NEQ = r'!='
t_GT = r'>'
t_LT = r'<'

# Símbolos
t_LBRACE   = r'\{'
t_RBRACE   = r'\}'
t_LBRACKET = r'\['
t_RBRACKET = r'\]'
t_LPAREN   = r'\('
t_RPAREN   = r'\)'
t_COLON    = r':'
t_DASH     = r'-'

# String: "algo"
def t_STRING(t):
    r'\"([^\\\n]|(\\.))*?\"'
    t.value = t.value[1:-1]  # quitar comillas
    return t

# Números
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

# Identificadores y palabras reservadas
def t_ID(t):
    r'[a-zA-Z_][a-zA-Z0-9_]*'
    t.type = reserved.get(t.value.upper(), 'ID')  
    return t

# Ignorar espacios y tabs
t_ignore = ' \t'

# Contar líneas
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# Errores
def t_error(t):
    print(f"Caracter ilegal: {t.value[0]}")
    t.lexer.skip(1)

# -----------------------
# Construcción del lexer
# -----------------------
lexer = lex.lex()

# -----------------------
# Prueba
# -----------------------
data = '''
GRUPO webservers {
    SERVIDOR "web01" DIRECCION = "192.168.1.10"
    SERVIDOR "web02" DIRECCION = "192.168.1.11"
}

MIENTRAS (VERIFICAR puerto != 80) HACER {
    EJECUTAR "reiniciar nginx"
}

SI (VERIFICAR NO_EXISTE archivo_conf) HACER {
    EJECUTAR ARCHIVO archivo_conf
}

VAR src = "/tmp"
VAR dest = "/bkp"
VAR archivo_conf = "/etc/config.conf"

'''

lexer.input(data)
for tok in lexer:
    print(tok)

## Tabla de Símbolos

In [ ]:
class SymbolTable:
    def __init__(self):
        self.variables = {}
        self.groups = {}
        self.servers = {}

    # --- Variables ---
    def add_variable(self, name, value):
        self.variables[name] = value
        print(f"[VAR] {name} = {value}")

    def resolve_value(self, token):
        if token in self.variables:
            return self.variables[token]
        return token

    # --- Grupos y servidores ---
    def add_group(self, name, servers):
        self.groups[name] = [s["nombre"] for s in servers]
        print(f"[GRUPO] {name} con {len(servers)} servidores")

    def add_server(self, name, direccion):
        self.servers[name] = direccion
        print(f"  [SERVIDOR] {name} @ {direccion}")

    def check_group_exists(self, name):
        if name not in self.groups:
            print(f"[Error Semántico] Grupo '{name}' no definido.")

## Parser

In [ ]:
import ply.yacc as yacc
from lexer import tokens
from semantic import SymbolTable

symtab = SymbolTable()
ast = []

# -----------------------
# GRAMÁTICA PRINCIPAL
# -----------------------

def p_programa(p):
    '''programa : definiciones instrucciones'''
    p[0] = ("programa", p[1], p[2])
    global ast
    ast = p[0]

# -----------------------
# DEFINICIONES DE GRUPOS
# -----------------------
def p_definiciones(p):
    '''definiciones : definiciones definicion
                    | definicion
                    | empty'''
    if len(p) == 3:
        p[0] = p[1] + [p[2]]
    elif len(p) == 2 and p[1] is not None:
        p[0] = [p[1]]
    else:
        p[0] = []

def p_definicion(p):
    '''definicion : GRUPO ID LBRACE servidores RBRACE'''
    p[0] = ("grupo", p[2], p[4])
    symtab.add_group(p[2], p[4])

def p_servidores(p):
    '''servidores : servidores servidor
                  | servidor'''
    if len(p) == 3:
        p[0] = p[1] + [p[2]]
    else:
        p[0] = [p[1]]

def p_servidor(p):
    '''servidor : SERVIDOR STRING DIRECCION EQUALS STRING'''
    p[0] = {"nombre": p[2], "direccion": p[5]}
    symtab.add_server(p[2], p[5])

# -----------------------
# INSTRUCCIONES
# -----------------------
def p_instrucciones(p):
    '''instrucciones : instrucciones instruccion
                     | instruccion
                     | empty'''
    if len(p) == 3:
        p[0] = p[1] + [p[2]]
    elif len(p) == 2 and p[1] is not None:
        p[0] = [p[1]]
    else:
        p[0] = []

def p_instruccion(p):
    '''instruccion : despliegue
                   | asignacion
                   | bucle
                   | condicional'''
    p[0] = p[1]

# -----------------------
# VARIABLES
# -----------------------
def p_asignacion(p):
    '''asignacion : VAR ID EQUALS valor'''
    symtab.add_variable(p[2], p[4])
    p[0] = ("var", p[2], p[4])

def p_valor(p):
    '''valor : STRING
             | NUMBER'''
    p[0] = p[1]

# -----------------------
# DESPLIEGUE
# -----------------------
def p_despliegue(p):
    '''despliegue : DESPLEGAR ID EN ID LBRACE acciones RBRACE'''
    p[0] = ("desplegar", p[2], p[4], p[6])
    symtab.check_group_exists(p[4])

def p_acciones(p):
    '''acciones : acciones accion
                | accion'''
    if len(p) == 3:
        p[0] = p[1] + [p[2]]
    else:
        p[0] = [p[1]]

# -----------------------
# ACCIONES (CORREGIDAS)
# -----------------------
def p_accion(p):
    '''accion : PAQUETE STRING DEBE_ESTAR_INSTALADO
              | SERVICIO STRING DEBE_ESTAR_EN_EJECUCION
              | COPIAR_DESDE origen HACIA destino
              | EJECUTAR STRING
              | EJECUTAR ARCHIVO ID'''
    
    if p[1] == "COPIAR_DESDE":
        p[0] = ("copiar", p[2], p[4])
    elif p[1] == "PAQUETE":
        p[0] = ("paquete", p[2])
    elif p[1] == "SERVICIO":
        p[0] = ("servicio", p[2])
    elif p[1] == "EJECUTAR":
        if len(p) == 3:
            p[0] = ("ejecutar", p[2])
        else:
            # EJECUTAR ARCHIVO ID
            p[0] = ("ejecutar_archivo", p[3])

def p_origen(p):
    '''origen : STRING
              | ID'''
    p[0] = symtab.resolve_value(p[1])

def p_destino(p):
    '''destino : STRING
               | ID'''
    p[0] = symtab.resolve_value(p[1])

# -----------------------
# CONDICIONAL
# -----------------------
def p_condicional(p):
    '''condicional : SI LPAREN VERIFICAR condicion RPAREN HACER LBRACE acciones RBRACE
                   | SI LPAREN VERIFICAR condicion RPAREN HACER LBRACE acciones RBRACE SINO LBRACE acciones RBRACE'''
    if len(p) == 10:
        p[0] = ("si", p[4], p[8])
    else:
        p[0] = ("si_sino", p[4], p[8], p[12])

# -----------------------
# BUCLE
# -----------------------
def p_bucle(p):
    '''bucle : MIENTRAS LPAREN VERIFICAR condicion RPAREN HACER LBRACE acciones RBRACE'''
    p[0] = ("mientras", p[4], p[8])

# -----------------------
# CONDICIONES
# -----------------------
def p_condicion(p):
    '''condicion : ID NEQ valor
                 | ID EQ valor
                 | ID GT valor
                 | ID LT valor
                 | NO_EXISTE ID'''
    if len(p) == 4:
        p[0] = (p[2], p[1], p[3])
    else:
        p[0] = ("no_existe", p[2])

# -----------------------
# VACÍO / ERROR
# -----------------------
def p_empty(p):
    'empty :'
    pass

def p_error(p):
    if p:
        print(f"Error de sintaxis en '{p.value}'")
    else:
        print("Error de sintaxis: fin inesperado")

parser = yacc.yacc()

def parse_code(data):
    parser.parse(data)
    return ast

## Executor

In [ ]:
def ejecutar(ast, symtab):
    print("\n--- EJECUCIÓN ---")
    for nodo in ast[2]:
        tipo = nodo[0]

        if tipo == "desplegar":
            nombre_app = nodo[1]
            grupo = nodo[2]
            acciones = nodo[3]
            servidores = symtab.groups.get(grupo, [])
            print(f"[Desplegando] {nombre_app} en grupo {grupo}: {servidores}")
            for accion in acciones:
                ejecutar_accion(accion)

        elif tipo == "mientras":
            condicion = nodo[1]
            acciones = nodo[2]
            print(f"[MIENTRAS] {condicion}")
            ejecutar_bucle(condicion, acciones)

        elif tipo in ("si", "si_sino"):
            condicion = nodo[1]
            acciones = nodo[2]
            print(f"[SI] {condicion}")
            ejecutar_condicional(condicion, acciones)


def ejecutar_accion(accion):
    tipo = accion[0]

    if tipo == "copiar":
        print(f"  Copiando de {accion[1]} hacia {accion[2]}")

    elif tipo == "paquete":
        print(f"  Asegurando paquete '{accion[1]}' instalado")

    elif tipo == "servicio":
        print(f"  Asegurando servicio '{accion[1]}' en ejecución")

    elif tipo == "ejecutar":
        print(f"  Ejecutando comando: {accion[1]}")

    elif tipo == "ejecutar_archivo":
        print(f"  Ejecutando archivo: {accion[1]}")


def ejecutar_bucle(condicion, acciones):
    print(f"    (Evaluar condición: {condicion})")
    for _ in range(2):  # Simula 2 iteraciones
        for accion in acciones:
            ejecutar_accion(accion)


def ejecutar_condicional(condicion, acciones):
    print(f"    (Evaluar condición: {condicion})")
    for accion in acciones:
        ejecutar_accion(accion)

## Main

In [ ]:
from parser import parse_code, symtab
from executor import ejecutar

data = '''
GRUPO webservers {
    SERVIDOR "web01" DIRECCION = "192.168.1.10"
    SERVIDOR "web02" DIRECCION = "192.168.1.11"
}

MIENTRAS (VERIFICAR puerto != 80) HACER {
    EJECUTAR "reiniciar nginx"
}

SI (VERIFICAR NO_EXISTE archivo_conf) HACER {
    EJECUTAR ARCHIVO archivo_conf
}

VAR src = "/tmp"
VAR dest = "/bkp"
VAR archivo_conf = "/etc/config.conf"

'''

ast = parse_code(data)
ejecutar(ast, symtab)